In [8]:
import sys
sys.path.append("/sps/lsst/users/ebarroso/crow")
from crow.cluster_modules.shear_profile import *

In [9]:
import pyccl as ccl
hmf = ccl.halos.MassFuncTinker08(mass_def="200c")
cosmo = ccl.Cosmology(
    Omega_c=0.2607,      # Cold dark matter density
    Omega_b=0.04897,     # Baryon density
    h=0.6766,            # Hubble parameter
    sigma8=0.8102,       # Matter fluctuation amplitude
    n_s=0.9665,          # Spectral index
)
cl_delta_sigma = ClusterShearProfile(cosmo, hmf, 4.0, True)

cl_reduced_shear = ClusterShearProfile(cosmo, hmf, 4.0, False)

cl_reduced_shear_interp = ClusterShearProfile(cosmo, hmf, 4.0, False, True)

cl_reduced_shear_interp_mis = ClusterShearProfile(cosmo, hmf, 4.0, False, False)


### Test values of shear profile

In [10]:
import time

start = time.time()
ds_val = cl_delta_sigma.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Delta sigma:", ds_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear.set_beta_parameters(10)
gt_val = cl_reduced_shear.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear exact:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp.set_beta_parameters(10)
cl_reduced_shear_interp.set_beta_s_interp(1.1, 1.3)
gt_val = cl_reduced_shear_interp.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta:", gt_val, " | Time:", end - start, "s")

start = time.time()
cl_reduced_shear_interp_mis.set_beta_parameters(10)
cl_reduced_shear_interp_mis.set_beta_s_interp(1.1, 1.3)
cl_reduced_shear_interp_mis.set_miscentering(0.2)
gt_val = cl_reduced_shear_interp_mis.compute_shear_profile([13], [1.2], 3.0)
end = time.time()
print("Reduced shear interpolated beta with miscentering:", gt_val, " | Time:", end - start, "s")


Delta sigma: [1.13451921e+12]  | Time: 0.002431631088256836 s
Reduced shear exact: [0.00027987]  | Time: 0.0990762710571289 s
Reduced shear interpolated beta: [0.00027987]  | Time: 0.04607439041137695 s
Reduced shear interpolated beta with miscentering: [0.00022393]  | Time: 0.16198039054870605 s


## Test implementation on the recipe

In [17]:
from crow.recipes.murata_binned_spec_z import MurataBinnedSpecZRecipe
from crow.cluster_modules.mass_proxy import MurataBinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
#from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataBinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()

def run_recipe(label, cluster_theory, mass_proxy_interval1=(1.1, 1.2), mass_proxy_interval2=(0.2, 0.31), redshiftinterval1=(0.9, 1.3), redshiftinterval2=(0.46, 0.76)):
    print(f"\n{label} Starts here\n{'-'*50}")
    average_on = ClusterProperty.NONE
    average_on |= ClusterProperty.DELTASIGMA
    recipe = MurataBinnedSpecZRecipe(
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
    )
    t0 = time.time()
    val1 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval1, redshiftinterval1, 1.0, 440, average_on)
    t1 = time.time()
    print(f"Eval 1: {val1:.3e} | Time: {t1-t0:.4f} s")

    t2 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on)
    t3 = time.time()
    print(f"Eval 2: {val2:.3e} | Time: {t3-t2:.4f} s")
    t22 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile_grid(mass_proxy_interval2, redshiftinterval2, 4.0, 440, average_on, mass_p=30, z_p=30)
    t33 = time.time()
    print(f"Eval 3 with grid: {val2:.3e} | Time: {t33-t22:.4f} s")
    t4 = time.time()
    counts = recipe.evaluate_theory_prediction_counts(mass_proxy_interval2, redshiftinterval2, 440)
    t5 = time.time() 
    print(f"Counts: {counts:.3e} Time: {t5-t4:.4f} s")
    t6 = time.time()   
    counts = recipe.evaluate_theory_prediction_counts_grid(mass_proxy_interval2, redshiftinterval2, 440, mass_p=20, z_p=20)
    t7 = time.time()
    print(f"Counts grid: {counts:.3e} Time: {t7-t6:.4f} s")


# Run different scenarios
run_recipe("Delta Sigma", cl_delta_sigma)
run_recipe("Tangential Shear", cl_reduced_shear)
run_recipe("Tangential Shear with Interpolation", cl_reduced_shear_interp)



Delta Sigma Starts here
--------------------------------------------------
Eval 1: 1.442e+16 | Time: 0.2900 s
Eval 2: 4.393e+15 | Time: 0.2880 s
Eval 3 with grid: 4.414e+15 | Time: 1.9678 s
Counts: 1.937e+03 Time: 0.0023 s
Counts grid: 1.805e+03 Time: 0.0411 s

Tangential Shear Starts here
--------------------------------------------------
Eval 1: 3.724e+00 | Time: 9.7268 s
Eval 2: 1.490e+00 | Time: 16.1062 s
Eval 3 with grid: 1.497e+00 | Time: 23.4610 s
Counts: 1.937e+03 Time: 0.0022 s
Counts grid: 1.805e+03 Time: 0.0409 s

Tangential Shear with Interpolation Starts here
--------------------------------------------------
Eval 1: 3.724e+00 | Time: 0.4916 s
Eval 2: 1.490e+00 | Time: 0.5211 s
Eval 3 with grid: 1.497e+00 | Time: 2.1780 s
Counts: 1.937e+03 Time: 0.0021 s
Counts grid: 1.805e+03 Time: 0.0412 s


In [21]:
from crow.recipes.murata_binned_spec_z_grid import MurataBinnedSpecZRecipeGrid
from crow.cluster_modules.mass_proxy import MurataUnbinned
from crow.cluster_modules.kernel import SpectroscopicRedshift
# from firecrown.models.cluster import ClusterProperty
from crow.properties import ClusterProperty
import time

pivot_mass, pivot_redshift = 14.625862906, 0.6
mass_distribution = MurataUnbinned(pivot_mass, pivot_redshift)
redshift_distribution = SpectroscopicRedshift()


def run_recipe(
    label,
    cluster_theory,
    mass_proxy_interval1=(1.1, 1.2),
    mass_proxy_interval2=(0.2, 0.31),
    redshiftinterval1=(0.9, 1.3),
    redshiftinterval2=(0.46, .76),
    sky_area=440.0,
):
    print(f"\n{label} Starts here\n{'-'*50}")

    average_on = ClusterProperty.NONE
    average_on |= ClusterProperty.DELTASIGMA

    recipe = MurataBinnedSpecZRecipeGrid(
        # mass_interval=(12.5, 15.5),
        # true_z_interval=(0.46, 1.3)ass_j in enumerate(self.log_mass_points):
                grid_2d[i, j] = self.completeness_distribution(np.array([log_mass_j]), np.array([z_i]))
,
        # proxy_interval=(0.2, 1.2),
        cluster_theory=cluster_theory,
        redshift_distribution=redshift_distribution,
        mass_distribution=mass_distribution,
    richness_points = 50,
    redshift_points= 50,
    log_mass_points= 50,
    )
    t0 = time.time()
    # Precompute all grids once
    recipe.recompute_all_grids(sky_area, radius_centers=[1.0, 4.0])
    t1 = time.time()
    print(t0 - t1)
    # ---- Shear 1 ----
    t0 = time.time()
    val1 = recipe.evaluate_theory_prediction_shear_profile(
        z_edges=redshiftinterval1,
        mass_proxy_edges=mass_proxy_interval1,
        radius_center=1.0,
        average_on=average_on,
    )
    t1 = time.time()
    print(f"Eval 1 (radius 1.0): {val1:.3e} | Time: {t1 - t0:.4f} s")

    # ---- Shear 2 ----
    t2 = time.time()
    val2 = recipe.evaluate_theory_prediction_shear_profile(
        z_edges=redshiftinterval2,
        mass_proxy_edges=mass_proxy_interval2,
        radius_center=4.0,
        average_on=average_on,
    )
    t3 = time.time()
    print(f"Eval 2 (radius 4.0): {val2:.3e} | Time: {t3 - t2:.4f} s")

    # ---- Counts ----
    t4 = time.time()
    counts = recipe.evaluate_theory_prediction_counts(
        z_edges=redshiftinterval2,
        mass_proxy_edges=mass_proxy_interval2,
        average_on=average_on,
    )
    t5 = time.time()
    print(f"Counts: {counts:.3e} | Time: {t5 - t4:.4f} s")


# Run different scenarios
run_recipe("Delta Sigma", cl_delta_sigma)
run_recipe("Tangential Shear", cl_reduced_shear)
run_recipe("Tangential Shear with Interpolation", cl_reduced_shear_interp)



Delta Sigma Starts here
--------------------------------------------------
-14.874666929244995
Eval 1 (radius 1.0): 3.541e+15 | Time: 0.0024 s
Eval 2 (radius 4.0): 5.716e+15 | Time: 0.0012 s
Counts: 5.210e+03 | Time: 0.0010 s

Tangential Shear Starts here
--------------------------------------------------
-109.79674243927002
Eval 1 (radius 1.0): 9.693e-01 | Time: 0.0026 s
Eval 2 (radius 4.0): 2.053e+00 | Time: 0.0012 s
Counts: 5.210e+03 | Time: 0.0010 s

Tangential Shear with Interpolation Starts here
--------------------------------------------------
-15.990453958511353
Eval 1 (radius 1.0): 1.488e+00 | Time: 0.0018 s
Eval 2 (radius 4.0): 2.151e+00 | Time: 0.0012 s
Counts: 5.210e+03 | Time: 0.0010 s


3.7772853185595565

1.7277486910994766